In [1]:
#importing various packages that we will use
import pandas as pd
import numpy as np
import os
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from statsmodels.tools.tools import add_constant
from scipy.optimize import minimize
from scipy.optimize import Bounds
import pdb
import datetime as dt
import matplotlib.pyplot as plt

param = {'dv01': {'x_SPX': 0, 'x_Oil': 0, 'x_DXY': 0,'x_UST_10Y': 925,'x_VIX': 0, 
                      'x_IG_Spread': 750, 'x_HY_Spread': 400,'y_EM_Spread': 5},
         'carry': {'x_SPX': 1.9, 'x_Oil': 0, 'x_DXY': 0,'x_UST_10Y': 3.0,'x_VIX': 0, 
                   'x_IG_Spread': 3.6, 'x_HY_Spread': 5.4, 'y_EM_Spread': 5.2}, 
         'shift': {'x_SPX': 0, 'x_Oil': 0, 'x_DXY': 0,'x_UST_10Y': 0,'x_VIX': 0, 
                   'x_IG_Spread': 0, 'x_HY_Spread': 0, 'y_EM_Spread': 0}}

# test_dir = r'C:\Users\Home\Documents\GitHub\Spread_Model_2.0
test_dir = r'C:\Users\chand\Projects\Python\Spread_Model\Spread_Model'

dt_today = dt.datetime.today().strftime('%Y%m%d')

In [10]:
def read_data(fl_name, periods): #setting default filename to be used unless specified
    fl = os.path.join(test_dir, fl_name) #joining directory and filename
    df = pd.read_csv(fl, header = 0, parse_dates = True) #reading csv file from source into a dataframe
    
    df.columns = ['Date', 'x_SPX', 'x_Oil', 'x_DXY', 'x_UST_10Y', 'x_VIX', 
                  'y_EM_Spread', 'x_IG_Spread', 'x_HY_Spread'] # setting column names for df
    
    df.drop(['x_IG_Spread', 'x_SPX', 'x_UST_10Y'], axis = 1, inplace = True)

    df.loc[:, 'Date'] = pd.to_datetime(df['Date']) #converting date into datetime format for pandas
#     df.loc[:, 'x_SPX'] = df['x_SPX'].astype('int64') #coverting SPX data to int
    cols = [col for col in df.columns if '_' in col] #filtering all columns with data
    df.loc[:, 'EM_Spread_raw'] = df['y_EM_Spread']
    df.loc[:, 'EM_Spread_prev'] = df['EM_Spread_raw'].shift(periods['discrete'] - periods['shift'])
    df.loc[:, 'y_EM_Spread'] = df['y_EM_Spread'].shift(-periods['shift'])
    df1 = periodic_change(df, cols = cols, periods = periods)
    df2 = periodic_total_return(df1, cols = cols, periods = periods) #calculating daily total return using custom function

#     df2.dropna(axis = 0, inplace = True) #remove day1 data (no change, return data avl)
    
    return df2

def periodic_change(df, cols, periods):
    for col in df[cols]:
        df.loc[:, col + '_chg'] = df[col].diff(periods = periods['discrete']) #calculating change given period

    return df


def periodic_total_return(df, cols, periods, ov_period = None):
    if ov_period is None:
        period_carry = periods['discrete']
    else:
        period_carry = ov_period
    for col in df[cols]:
        df.loc[:, col + '_carry_rtn'] = param['carry'][col] / 100 / 360 * period_carry #Periodic Carry
        dv01 = param['dv01'][col]
        if dv01 == 0:
            df.loc[:, col + '_price_rtn'] = df[col].pct_change(periods = periods['discrete']) #Pct Change for price based inputs
        if dv01 != 0:
            df.loc[:, col + '_price_rtn'] = df[col].diff(periods = periods['discrete']) * -1 * dv01 / 10000 #Spread in bp * Duration converted to a %age of 1
        df.loc[:, col + '_total_rtn'] = df.loc[:, col + '_carry_rtn'] + df.loc[:, col + '_price_rtn'] # Total Return = Carry + Px Return
    
    return df

In [11]:
def reg_m(df):
    f, col_y, col_x = formula_create(df)
#     print (f, '\n', col_y, '\n', (df.loc[:, [col for col in df.columns if '_total_rtn' in col]].head()))
    model = smf.ols(formula = f, data=df)
    results = model.fit()
    
    df_fitval = pd.DataFrame({'y_EM_Spread_pred': results.fittedvalues})
    df_resid = pd.DataFrame({'y_EM_Spread_resid': results.resid})
    
    df = df.merge(df_fitval, how='outer', left_index=True, right_index=True)
    df = df.merge(df_resid, how='outer', left_index=True, right_index=True)
    
    df_vif = pd.DataFrame()
    if typ_model == 'static':
        df_vif_input = df.loc[:, col_x]
        X = df_vif_input.dropna()

        df_vif = pd.DataFrame({'Variables': X.columns,
                               'VIF_Values': [vif(X.values, i) for i in range(X.shape[1])]}).set_index('Variables')
    return results, df, df_vif, col_x

def formula_create(df):
    if typ == 'chg':
        cols_x = [col for col in df.columns if ('_chg' in col) & ('x_' in col[:2])]
        cols_y = [col for col in df.columns if ('_chg' in col) & ('y_' in col[:2])]
        formula_text = '{} ~ {} - 1'.format(cols_y[0], ' + '.join(cols_x)) #dropping intercept
        
    elif typ == 'total_rtn':
        cols_x = [col for col in df.columns if ('_total_rtn' in col) & ('x_' in col[:2])]
        cols_y = [col for col in df.columns if ('_total_rtn' in col) & ('y_' in col[:2])]
        formula_text = '{} ~ {} - 1'.format(cols_y[0], ' + '.join(cols_x)) #dropping the constant
        
    elif typ == 'abs':
        cols = [col for col in df.columns if ('_rtn' not in col) & ('_chg' not in col)]
        cols_x = [col for col in cols if 'x_' in col[:2]]
        cols_y = [col for col in cols if 'y_' in col[:2]]
        formula_text = '{} ~ {} - 1'.format(cols_y[0], ' + '.join(cols_x)) #adding regressor terms and dropping the intercept
    
    return formula_text, cols_y, cols_x

In [12]:
def plot_reg(results, df):
    Y = 'y_EM_Spread'
#     cell_text = []
#     cell_text.append([results.params])
    
    fig, ax = plt.subplots(figsize=(18, 10))
    ax.plot(df['Date'], df[Y], color='green', label = Y[2:], linewidth = 0.6)
    ax.set_ylabel(Y, fontsize=16, weight='bold', va='top')
    ax.plot(df['Date'], df[Y + '_pred_val'], color='orange', label = Y[2:] + ' Predict', linewidth = 1)
    
    ax.set_xlabel("Date", fontsize=16, weight='bold', va='top')
    ax.set_title("Linear Regression", fontsize=16, weight='bold', va='top')
    
    ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis    
    ax2.plot(df['Date'], df[Y + '_resid_val'], color='blue', label = 'Residuals', linewidth= 0.6, ls = 'dashed')
    ax2.set_ylabel('Residual Spread', fontsize=16, weight='bold', va='top')
    
    ax.legend(loc='upper left')
    f = 'R Squared = ' + str(round(results.rsquared,3))
    ax.text(0.8, 0.95, f, horizontalalignment='center', verticalalignment='center',
            fontsize=16, weight='bold', va='top', transform=ax.transAxes)
#     plt.show()
#     print (results.params)
    return plt

In [13]:
def save_output(df, res):
    fl_name = 'Spread_Model_Output_{}.csv'.format(dt_today)
    fl = os.path.join(test_dir, fl_name) #joining directory and filename
    df.to_csv(fl, index = False)
    
    fl_name = 'Spread_Model_OLS_{}.txt'.format(dt_today)
    fl = os.path.join(test_dir, fl_name)
    text_file = open(fl, "w")
    text_file.write(res.summary().as_text())
    text_file.close()
    
    return

In [19]:
def params_to_df(df, res, cols, Y = 'y_EM_Spread'):
    df_res = pd.DataFrame(columns = ['Variable', 'Co-eff']).set_index('Variable')
    for col in cols:
        if (typ == 'abs') | (typ_calc == 'predict'):
            df_col = pd.DataFrame({'Variable': [col], 'Co-eff': [res.params[col]]}).set_index('Variable')
            df.loc[:, col + '_coeff'] = res.params[col]
            df.loc[:, Y + '_pred_v2'] += df[col] * df[col + '_coeff']
        
        else:
            col1 = col + '_' + typ
            df_col = pd.DataFrame({'Variable': [col1], 'Co-eff': [res.params[col1]]}).set_index('Variable')
            df[col1 + '_coeff'] = res.params[col1]
            df.loc[:, Y + '_pred_v2'] += df[col1] * df[col1 + '_coeff']
#         print (df_col)
        df_res = df_res.append(df_col)
#     print (df, df_res)
#     pdb.set_trace()
    return df, df_res
    
def calc_sprd(res, df, cols, typ_calc = 'predict', df_inputs = pd.DataFrame(), ov_period = None):
    Y = 'y_EM_Spread'
#     print (cols)
    y_last = {'Raw': 0, 'Predict': 0, 'Shock': 0}
    y_predict = 0
    df.loc[:, Y + '_pred_v2'] = 0
    
    if res is not None:
        df, df_res = params_to_df(df, res, cols, Y)

    if typ == 'total_rtn':
        df.loc[:, Y + '_pred_v2'] -= df[Y + '_carry_rtn']   
    
    if typ_calc == 'predict':
        y_last['Raw'] = df.iloc[-1:][Y[2:] + '_raw'].item()
        df2 = df.iloc[-2:]
        
    elif (typ_calc == 'shock') & (len(df_inputs) != 0):
        y_last['Raw'] = df.iloc[-1:][Y[2:] + '_raw'].item()
        df1 = pd.concat([df, df_inputs], join = 'outer', sort = 'False')

        if typ == 'abs':
            df2 = df1.iloc[-2:].loc[:, cols]
            for col in cols:
                y_predict += df_res.loc[col, 'Co-eff'] * df2.iloc[-1:][ col]
                
        if typ == 'chg':
            df2 = periodic_change(df1.iloc[-2:].loc[:, cols], cols = cols, periods = 1)
            for col in cols:
                y_predict += df_res.loc[col + '_chg', 'Co-eff'] * df2.iloc[-1:][col + '_chg'].item()
                
        if typ == 'total_rtn': 
            df2 = periodic_total_return(df1.iloc[-2:].loc[:, cols], cols = cols, periods = 1, ov_period = ov_period)
            y_carry = param['carry'][Y] / 100 / 360 * ov_period
            y_predict -= y_carry
            for col in cols:
                y_predict += df_res.loc[col + '_total_rtn', 'Co-eff'] * df2.iloc[-1:][col + '_total_rtn'].item()
    else:
        print ('Type Calc Error!')
        exit()
    
    if typ == 'abs':
        df.loc[:, Y + '_pred_val'] = df[Y + '_pred_v2']
        y_last['Shock'] = y_predict
        
    elif typ == 'chg':
        df.loc[:, Y + '_pred_val'] = df[Y[2:] + '_prev'] + df[Y + '_pred_v2']
        y_last['Shock'] = y_last['Raw'] + y_predict
        
    elif typ == 'total_rtn':
        df.loc[:, Y + '_pred_val'] = df[Y[2:] + '_prev'] + df[Y + '_pred_v2'] / param['dv01'][Y] * 10000 * -1
        y_last['Shock'] = y_last['Raw'] + y_predict / param['dv01'][Y] * 10000 * -1
    
    df.loc[:, Y + '_resid_val'] = df[Y] - df[Y + '_pred_val']
    df.loc[:, Y + '_resid_val_v2'] = abs(df[Y] - df[Y + '_pred_val'])
    y_last['Predict'] = df.iloc[-1:][Y + '_pred_val'].item()
    
    df.loc[:, Y[2:] + '_Actual'] = (df[Y[2:] + '_prev'] - df[Y]) \
                                    * param['dv01'][Y] / 10000
    df.loc[:, Y + '_resid_v2'] = abs(df[Y[2:] + '_Actual'] - df[Y + '_pred_v2'])
    df.loc[:, 'Pred_dir'] = np.where(df[Y[2:] + '_Actual'] / df[Y + '_pred_v2'] > 0, 1, 0)
    df.loc[:, Y + '_chg_abs'] = df[Y + '_chg'].abs()
    
#     print (df.tail())
#     pdb.set_trace()
    if typ_calc == 'predict':
        print ('Current Value vs Model Values :')
        print (Y[2:], ' : ', y_last['Raw'], '\n', Y[2:], ' Predict : ', round(y_last['Predict'], 1),  '\n')
#         print ('Changes :', '\n', df2)
        print ('Direction of predictions :', '\n', df.groupby('Pred_dir').agg({'Date': 'count', 
                                                                               Y + '_resid_v2': np.std,
                                                                               Y + '_resid_val_v2': np.std,
                                                                               Y + '_resid': np.std,
                                                                               Y + '_resid_val': np.std, 
                                                                               Y + '_chg_abs': sum}).round(3))
        print ('\n')
        
    elif typ_calc == 'shock':
        print ('Current Value vs Scenario Values :')
        print (Y[2:], ' : ', y_last['Raw'], '\n', Y[2:], ' Predict : ', round(y_last['Shock'], 1))
#         print ('Coeff : ', '\n', res.params)
        print ('Scenario : ', '\n', df2.loc[:, cols], '\n', 'Predicted Y: ', round(y_predict, 4), '\n\n\n')

    return y_last, df

In [41]:
def data_adj(df, periods):
    df.sort_values(['Date'], ascending = False, inplace = True) #sort dates in reverse chronological order
    df_discrete = df[:: (periods['discrete'] - periods['overlap'])].copy() #create a non-rolling series by selecting every 'period' data less 'overlap' if allowed
    df_discrete.sort_values(['Date'], ascending = True, inplace = True) #sort dates in chronological order and save into same variable

    df_reg = df_discrete[periods['reg']['start']: periods['reg']['end']].copy() #slicing dataset to keep only the 'last' datapoints
    df_pred = df_discrete[periods['pred']['start']: periods['pred']['end']].copy()
    
    return df_reg, df_pred

def model_static(df_reg, df_pred):

    res, df_reg_output, vifs, cols = reg_m(df_reg)
    df_pred['y_EM_Spread_pred'], df_pred['y_EM_Spread_resid'] = 0, 0
    y, df_pred_output = calc_sprd(res, df_pred, cols)
    
    return res, df_pred_output, df_reg_output, vifs

def model_scenario(res, df):
    #Shock / Scenario Analysis
    inputs = {'x_VIX': [22], 
              'x_Oil': [55], 
              'x_DXY': [98],
              'x_HY_Spread': [4.25]}
    ov_period = 30 #calendar days
    df_inputs = pd.DataFrame.from_dict(inputs) #convert dict into df
    cols = [col for col in df_inputs.columns] #make a list of all columns in inputs
    # print (cols)
    y, df_scenario_output = calc_sprd(res, df, cols, typ_calc = 'shock', df_inputs = df_inputs, ov_period = ov_period)
    return y, df_scenario_output

def model_rolling(df, periods):
    df_results = pd.DataFrame()
    Y = 'y_EM_Spread'
    df.loc[:, Y + '_pred_v2'] = 0
    for idx in range(periods['rolling'], len(df[::(periods['discrete']- periods['overlap'])])):
        periods['reg']['start'] = idx - periods['rolling']
        periods['reg']['end'] = idx
        periods['pred']['start'] = idx
        periods['pred']['end'] = idx + 1
        df_reg, df_pred = data_adj(df, periods)
        try:
            res, df_reg_output, vifs, cols = reg_m(df_reg)
        except (RuntimeError, TypeError, NameError, IndexError):
            print (idx, df_reg.shape, periods['reg'], df.shape)
            pdb.set_trace()
        
        df_current = df_reg_output.iloc[-1:].copy()
        df_current_results, df_current_coeff = params_to_df(df_current, res, cols, Y)
#         print (df_current_coeff)
        df_results = df_results.append(df_current_results)
    print ('Original : ', df.shape, '\n', 'Results : ', df_results.shape, '\n', 'Parameters : ', periods)
    y, df_results_final = calc_sprd(res = None, df = df_results, cols = cols)
    
    df_results.loc[:, Y] = df_results[Y].astype(float)
    df_results.loc[:, Y + '_pred_val'] = df_results[Y + '_pred_val'].astype(float)
#     print (df_results[Y].dtype, df_results[Y + '_pred_val'].dtype)
    results_corr = df_results[Y].corr(df_results[Y + '_pred_val'])
    print ('R-Squared : ', round(results_corr ** 2, 2))
    return y, df_results

In [42]:
periods = {'discrete': 15, 
           'rolling': 90, 
           'overlap': 0, 
           'last': 2000, 
           'shift': 0, 
           'reg': {'start': 0, 'end': 0}, 
           'pred': {'start': 0, 'end': -1}}
typ = 'total_rtn' #Types = abs, chg, total_rtn
typ_calc = 'predict' #Types = predict, shock
typ_model = 'rolling' #Types = static, rolling

df = read_data(fl_name = 'Spread_Model_Sample_Data.csv', periods = periods)

if typ_model == 'static':
    periods['reg']['end'] = int((len(df) - periods['last']) / (periods['discrete'] - periods['overlap']))
    periods['pred']['start'] = int((len(df) - periods['last']) / (periods['discrete'] - periods['overlap']))

    df_reg, df_pred = data_adj(df, periods)
    res, df_results, df_static_reg, vifs = model_static(df_reg = df_reg, df_pred = df_pred)
    plt = plot_reg(res, df_static_pred, typ)
    
elif typ_model == 'rolling':
    y, df_results = model_rolling(df = df, periods = periods)
else:
    print ('Type Model Error!')

# y, df_scenario = model_scenario(res, df_static_pred)

if typ_model == 'static':
    plt.show()
    print (res.summary(), '\n\n\n', 'Multi-Collinearity Contributions (Variance Inflation Factors): ','\n', vifs)
    save_output(df_results, res)


Original :  (4788, 29) 
 Results :  (230, 35) 
 Parameters :  {'discrete': 15, 'rolling': 90, 'overlap': 0, 'last': 2000, 'shift': 0, 'reg': {'start': 229, 'end': 319}, 'pred': {'start': 319, 'end': 320}}
Current Value vs Model Values :
EM_Spread  :  280 
 EM_Spread  Predict :  270.3 

Direction of predictions : 
           Date  y_EM_Spread_resid_v2  y_EM_Spread_resid_val_v2  \
Pred_dir                                                         
0          125                 0.009                    17.941   
1          105                 0.020                    39.762   

          y_EM_Spread_resid  y_EM_Spread_resid_val  y_EM_Spread_chg_abs  
Pred_dir                                                                 
0                     0.006                 17.941               2160.0  
1                     0.009                 40.161               2231.0  


R-Squared :  0.93


In [ ]:
fl_name = 'Spread_Model_Output_{}.csv'.format(dt_today)
fl = os.path.join(test_dir, fl_name) #joining directory and filename
df1.to_csv(fl, index = False)

In [ ]:
print (np.std(df2_2.loc[:, [col for col in df2.columns if '_total_rtn' in col]]))
print (np.std(df2_2.loc[:, [col for col in df2.columns if '_chg' in col]]))

In [ ]:

cols = ['Date', 'y_EM_Spread', 'EM_Spread_raw', 'EM_Spread_prev', 'y_EM_Spread_pred_val', 'y_EM_Spread_resid_val', 
        'y_EM_Spread_total_rtn', 'y_EM_Spread_pred', 'y_EM_Spread_resid', 'y_EM_Spread_pred_v2', 'y_EM_Spread_resid_v2',
        'EM_Spread_Actual', 'y_EM_Spread_carry_rtn']
print (df2_2.loc[:, cols].tail())
# cols = ['Date', 'EM_Spread_prev', 'y_EM_Spread_pred_val_v2', 'y_EM_Spread_pred_v2',
#         'x_SPX_total_rtn_coeff', 'x_SPX_total_rtn',
#         'x_Oil_total_rtn_coeff', 'x_Oil_total_rtn',
#         'x_DXY_total_rtn_coeff', 'x_DXY_total_rtn',
#         'x_UST_10Y_total_rtn_coeff', 'x_UST_10Y_total_rtn']
# print (df2_2.loc[:, cols].tail())

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [26]:
print (df_results.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 1337 to 4772
Data columns (total 42 columns):
Date                           230 non-null datetime64[ns]
x_Oil                          230 non-null float64
x_DXY                          230 non-null float64
x_VIX                          230 non-null float64
y_EM_Spread                    230 non-null int64
x_HY_Spread                    230 non-null float64
EM_Spread_raw                  230 non-null int64
EM_Spread_prev                 230 non-null float64
x_Oil_chg                      230 non-null float64
x_DXY_chg                      230 non-null float64
x_VIX_chg                      230 non-null float64
y_EM_Spread_chg                230 non-null float64
x_HY_Spread_chg                230 non-null float64
x_Oil_carry_rtn                230 non-null float64
x_Oil_price_rtn                230 non-null float64
x_Oil_total_rtn                230 non-null float64
x_DXY_carry_rtn                230 non-null float64
x_DX

In [ ]:
print (df2_2.tail())